In [1]:
from framework.platform import PlatformManager
from framework.configs import PlatformConfig, EvalConfig, FeatureExtractor
import os

eval_cfg = EvalConfig(
    prd=False,
    prd_plot=False,
    feature_extractor=FeatureExtractor.InceptionV3,
    c2st_knn=False,
    fid_infinity=False,
    is_infinity=False,
    clean_fid=False,
    clean_kid=False,
    inception_score=False,
    fid=False,
    kid=False,
    prc=False,
    ls=False,
    mifid=True,
)

platform_cfg = PlatformConfig(
    verbose=True,
    cuda=True,
    save_cpu_ram=True,
    compare_real_to_real=True,
    num_worker=16,
)

platform_manager = PlatformManager(real_images_path=os.path.join(os.getcwd(), "org_64_test"),
                                   eval_config=eval_cfg,
                                   platform_config=platform_cfg)

result_dict = platform_manager.calc_metrics()
result_dict.print()
result_dict.write_to_json(file="dict/miFID.json")

d:\Users\qrno9\miniconda3\envs\GIM-Eval\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


[INFO]: Real images source found, name:CelebA64 (Original)
[INFO]: 8 different generator sources found. Names:
a_tewst
b_test
CelebAHQ64
cifar-10
diffusionStyleGAN2
diffusionVAE
guidedDiffusion_IP
noise64
[INFO]: Comparison real-to-real (True)
[START]: Calculating Metrics for a_tewst
[INFO]: Start Calculation MiFID, Source = a_tewst
[INFO]: Start Calculation FID, Source = a_tewst


Creating feature extractor "inception-v3-compat" with features ['2048']
Extracting features from input1
Processing samples                                                           
Extracting features from input2
Processing samples                                                           
Frechet Inception Distance: 39.864466274300355
Kernel Inception Distance: 0.018253629684684705 ± 0.0006278361871706389         


[INFO]: FID finished
[INFO]: No precomputed real or generated features found. Compute from CleanFID
[INFO]: Start Calculation Clean FID, Source = a_tewst
[INFO]: Feature Extractor used: InceptionV3
compute FID between two folders
Found 1367 images in the folder d:\Dokumente\SoftwareDev\MasterThesis\GIM-evaluation\org_64_test


FID org_64_test : 100%|██████████| 43/43 [00:09<00:00,  4.38it/s]


...

[INFO]: FID finished
[INFO]: No precomputed real or generated features found. Compute from CleanFID
[INFO]: Start Calculation Clean FID, Source = b_test
[INFO]: Feature Extractor used: InceptionV3
compute FID between two folders
[INFO]: Use precomputed real features
Found 1027 images in the folder d:\Dokumente\SoftwareDev\MasterThesis\GIM-evaluation\generated_images\b_test


FID b_test : 100%|██████████| 33/33 [00:04<00:00,  6.85it/s]


[INFO]: Clean FID finished
[INFO]: Feature Extractor used: InceptionV3
torch.Size([1027, 1367])
[INFO]: MiFID m_tau = 1.0
[INFO]: MiFID finished
[FINISHED]: Calculating Metrics for b_test
{'Frechet Inception Distance': 427.20624096504804, 'Clean FID': 434.79285583030077, 'MiFID': 427.20624096504804}
[START]: Calculating Metrics for CelebAHQ64
[INFO]: Start Calculation MiFID, Source = CelebAHQ64
[INFO]: Start Calculation FID, Source = CelebAHQ64


Creating feature extractor "inception-v3-compat" with features ['2048']
Extracting features from input1
Processing samples                                                           
Extracting features from input2


KeyboardInterrupt: 

In [2]:
from framework.platform import PlatformManager
from framework.configs import PlatformConfig, EvalConfig, FeatureExtractor
import os


eval_cfg = EvalConfig(
    prd=True,
    prd_plot=True,
    feature_extractor=FeatureExtractor.VGGFaceResNet50,
    c2st_knn=True,
    fid_infinity=True,
    is_infinity=True,
    clean_fid=True,
    clean_kid=True,
    inception_score=True,
    fid=True,
    kid=True,
    prc=True,
    ls=True,
)

platform_cfg = PlatformConfig(
    verbose=True,
    cuda=True,
    save_cpu_ram=True,
    compare_real_to_real=True,
    num_worker=16,
)

platform_manager = PlatformManager(real_images_path=os.path.join(os.getcwd(), "original_64"),
                                   eval_config=eval_cfg,
                                   platform_config=platform_cfg)

[INFO]: Real images source found, name:CelebA64 (Original)
[INFO]: 6 different generator sources found. Names:
CelebAHQ64
cifar-10
diffusionStyleGAN2
diffusionVAE
guidedDiffusion_IP
noise64


In [68]:
import torch
r = torch.ones(20000, 2048)
g = torch.ones(10000, 2048)

from torchmetrics.functional import pairwise_cosine_similarity
#cos = CosineSimilarity(dim=1, eps=1e-6)
cos_distances = []
cos_sim = pairwise_cosine_similarity(g, r)
print(torch.min(1 - cos_sim, dim=1).values.mean().clamp(min=0.0, max=1.0))



tensor(5.9605e-08)


In [3]:

import torch 


real_imgs = []
for imgs in platform_manager.get_real_images_src().get_dataloader(batch_size=64, num_worker=20):
    real_imgs.append(imgs)
real_imgs = torch.cat(real_imgs)
print(real_imgs.size())
gen_imgs = []
for imgs in platform_manager.get_generator_src("guidedDiffusion_IP").get_dataloader(batch_size=64, num_worker=20):
    gen_imgs.append(imgs)
gen_imgs = torch.cat(gen_imgs)
print(gen_imgs.size())


torch.Size([202599, 3, 64, 64])
torch.Size([50000, 3, 64, 64])


In [6]:
from torchmetrics.image.mifid import MemorizationInformedFrechetInceptionDistance

device = "cuda" if torch.cuda.is_available() else "cpu"
mifid = MemorizationInformedFrechetInceptionDistance(feature=2048)
mifid.to(device)
mifid.update(real_imgs, real=True)
mifid.update(gen_imgs, real=False)
print(mifid.compute())

RuntimeError: [enforce fail at C:\cb\pytorch_1000000000000\work\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 217350638388 bytes.

In [3]:
from framework.platform import ResultDict

results = ResultDict({})
results.read_from_json("dict/all_noise_celebHQ_vggface_old.json")
results.print(round_scores=True)
#results.normalize_scores(print_results=True, exlude_IS=False)

------------------------------------
Generator Name: CelebAHQ64
------------------------------------
Inception Score Mean: 1.0
Inception Score Std: 0.0
Frechet Inception Distance: 2247.051
Kernel Inception Distance Mean: 5.213
Kernel Inception Distance Std: 0.171
Precision: 0.061
Recall: 0.93
F1 Score: 0.115
Clean FID: 2353.092
FID Infinity (Approx.): 2329.365
IS Infinity (Approx.): 1.0
Clean KID: 5.354
LS: 0.584
C2ST Adaptive KNN Accuracy: 0.523
C2ST Adaptive KNN Normalized: 0.953
PRD F8 Max Precision: 0.829
PRD F8 Max Recall: 0.766
------------------------------------
Generator Name: cifar-10
------------------------------------
Inception Score Mean: 1.0
Inception Score Std: 0.0
Frechet Inception Distance: 6116.579
Kernel Inception Distance Mean: 9.799
Kernel Inception Distance Std: 0.211
Precision: 0.374
Recall: 0.002
F1 Score: 0.004
Clean FID: 5833.203
FID Infinity (Approx.): 5826.039
IS Infinity (Approx.): 1.0
Clean KID: 9.195
LS: 0.554
C2ST Adaptive KNN Accuracy: 0.901
C2ST Adapt

In [1]:
from framework.util import img_to_64

img_to_64(path="D:/Dokumente/SoftwareDev/MasterThesis/CelebAMask-HQ/CelebAMask-HQ/CelebA-HQ-img/", 
          out_path="D:/Dokumente/SoftwareDev/MasterThesis/GIM-evaluation/generated_images/CelebAHQ64",
          npz=False)

(30000, 64, 64, 3) size array saved into celeba64_train.npz


In [1]:
from framework.util import create_noise64_imgs

create_noise64_imgs(out_path="D:/Dokumente/SoftwareDev/MasterThesis/GIM-evaluation/generated_images/noise64",
                    num_samples=50000)


[Create and Write Noise Imgs]:: 100%|##########| 50000/50000 [00:20<00:00, 2463.10it/s]
